In [ ]:
"""' Install Dependencies """

!pip install -U langchain langchain-openai langchain-deepseek langchain-qwq

In [ ]:
""" Setting Environment Variables """

import os
import json

try:
    from google.colab import drive

    drive.mount("/content/drive")
except Exception as e:
    print("Not in Colab or already mounted.")

with open("./drive/MyDrive/llm_agents/data/data_config.json", "r", encoding="utf-8") as f:
    data_config = json.load(f)

# ["gpt-4.1-2025-04-14", "o3-2025-04-16"]  For details, see https://platform.openai.com/docs/
os.environ["OPENAI_API_BASE"] = data_config.get("large_language_model").get("openai").get("openai_api_base")
os.environ["OPENAI_API_KEY"] = data_config.get("large_language_model").get("openai").get("openai_api_key")

# ["deepseek-chat", "deepseek-reasoner"]  For details, see https://api-docs.deepseek.com/
os.environ["DEEPSEEK_API_BASE"] = data_config.get("large_language_model").get("deepseek").get("deepseek_api_base")
os.environ["DEEPSEEK_API_KEY"] = data_config.get("large_language_model").get("deepseek").get("deepseek_api_key")

# ["llama3.1-70b-instruct", "qwen3-32b"]  For details, see https://bailian.console.aliyun.com/
os.environ["DASHSCOPE_API_BASE"] = data_config.get("large_language_model").get("aliyuncs").get("dashscope_api_base")
os.environ["DASHSCOPE_API_KEY"] = data_config.get("large_language_model").get("aliyuncs").get("dashscope_api_key")


In [ ]:
import os
import json
import time
import ast
from typing import Any
from pydantic import Field

import pandas as pd
from tqdm import tqdm

from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_qwq import ChatQwQ
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor

try:
    from google.colab import drive

    drive.mount("/content/drive")
except Exception as e:
    print("Not in Colab or already mounted.")

# ======================================================================
#                         Global Configuration
# ======================================================================
config = {}


def set_config(config_file_path, dataset_name):
    global config

    with open(config_file_path, "r", encoding="utf-8") as f:
        data_config = json.load(f)

    config = {
        "MODEL_NAME": "gpt-4.1-2025-04-14",

        "KNOWLEDGE_BASE_TEMPLATE": os.path.join(data_config.get("prompt").get("dpath"),
                                                data_config.get("prompt").get("knowledge_base_template")),
        "QK_PROMPT": os.path.join(data_config.get("prompt").get("dpath"), data_config.get("prompt").get("qk_prompt")),
        "KC_RANGE_PROMPT": os.path.join(data_config.get("prompt").get("dpath"),
                                        data_config.get("prompt").get("kc_range_prompt")),
        "KC_USE_PROMPT": os.path.join(data_config.get("prompt").get("dpath"),
                                      data_config.get("prompt").get("kc_use_prompt")),
        "QKC_PROMPT": os.path.join(data_config.get("prompt").get("dpath"), data_config.get("prompt").get("qkc_prompt")),

        "QUESTION_FILE_PATH": os.path.join(data_config.get("dataset").get(dataset_name).get("dpath"),
                                           data_config.get("dataset").get(dataset_name).get("question_file")),
        "KNOWLEDGE_TAGS_FILE_PATH": os.path.join(data_config.get("dataset").get(dataset_name).get("dpath"),
                                                 data_config.get("dataset").get(dataset_name).get("knowledge_file")),

        "QK_SAVE_PATH": os.path.join(data_config.get("dataset").get(dataset_name).get("save_path"),
                                     data_config.get("dataset").get(dataset_name).get("(llm agents) qk_file")),
        "KC_USE_SAVE_PATH": os.path.join(data_config.get("dataset").get(dataset_name).get("save_path"),
                                         data_config.get("dataset").get(dataset_name).get("(llm agents) kc_use_file")),
        "QKC_SAVE_PATH": os.path.join(data_config.get("dataset").get(dataset_name).get("save_path"),
                                      data_config.get("dataset").get(dataset_name).get("(llm agents) qkc_file"))
    }


# ======================================================================
#                          LLM Initialization
# ======================================================================
def set_llm(model_name="gpt-4.1-2025-04-14"):
    if model_name in ["gpt-4.1-2025-04-14"]:
        llm = ChatOpenAI(
            model=model_name,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
    elif model_name in ["deepseek-chat"]:
        llm = ChatDeepSeek(
            model=model_name,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
    elif model_name in ["llama3.1-70b-instruct", "qwen3-32b"]:
        llm = ChatQwQ(
            model=model_name,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
    else:
        raise ValueError(f"Invalid model name: {model_name}")

    return llm


# ======================================================================
#      Phase 1: Generating Knowledge Components and Their Weights
# ======================================================================
class IdentifyQuestionKnowledgeComponents(BaseTool):
    name: str = "IdentifyQuestionKnowledgeComponents"
    description: str = (
        "Use this tool to identify the 1 to 5 most relevant knowledge components associated with a given question. "
        "Assign a numeric weight (float, two decimal places) to each selected knowledge component, reflecting its relative importance. "
        "Ensure that the sum of all weights strictly equals 1.00. "
        "Provide the 'question_description' parameter, which represents the description of the question."
    )
    llm: Any = Field(default=None)

    def _run(self, question_description: str):
        with open(config.get("QK_PROMPT"), "r", encoding="utf-8") as file:
            qk_prompt = file.read()

        df_k = pd.read_csv(config.get("KNOWLEDGE_TAGS_FILE_PATH"))
        knowledge_tags = list(df_k['knowledge_component_name'])
        qk_prompt = qk_prompt.replace("{placeholder - domain knowledge components}",
                                      "{" + ', '.join(knowledge_tags) + "}")
        prompt = qk_prompt + "\n" + question_description

        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        return response


class Phase1Executor:
    def __init__(self, llm, role, task, background_knowledge, tools, final_answer_format, config, history_window=5):
        self.llm = llm
        self.role = role
        self.task = task
        self.background_knowledge = background_knowledge
        self.tools = tools
        self.tool_names = ", ".join([tool.name for tool in tools])
        self.final_answer_format = final_answer_format
        self.config = config

        with open(self.config.get("KNOWLEDGE_BASE_TEMPLATE"), "r", encoding="utf-8") as file:
            knowledge_base_template = file.read()

        self.prompt = PromptTemplate(
            input_variables=["role", "task", "background_knowledge", "tools", "tool_names", "final_answer_format",
                             "input", "agent_scratchpad"],
            template=knowledge_base_template,
            validate_template=False
        )

        self.agent_1 = create_react_agent(llm=self.llm, tools=self.tools, prompt=self.prompt)
        self.agent_executor_1 = AgentExecutor(agent=self.agent_1, tools=self.tools, verbose=True,
                                              handle_parsing_errors=True)

        self.history = []
        self.history_window = history_window

    # Return formatted string of previous m rounds (Q&A) for current prompt.
    def get_history(self):
        if not self.history:
            return ""

        history_lines = []
        for idx, rec in enumerate(self.history):
            history_lines.append(
                f"[History round {idx + 1}]\nQuestion (ID: {rec['q_id']}): {rec['input']}\nOutput: {rec['output']}\n"
            )
        return "\n".join(history_lines)

    # Get each question information.
    def load_data(self, q_file_path):
        data = pd.read_csv(q_file_path)
        q_ids = data["question_id"].tolist()
        q_texts = data["description"].tolist()

        return q_ids, q_texts

    # Save result ((llm agents_{model_name}) question_knowledge.csv).
    def save_data(self, q_id, response, save_file_path):
        df = pd.DataFrame([(q_id, response)], columns=["question_id", "knowledge_component"])
        header = not os.path.exists(save_file_path)
        df.to_csv(save_file_path, mode="a", header=header, index=False)

    def run(self):
        try:
            q_ids, q_texts = self.load_data(self.config.get("QUESTION_FILE_PATH"))
            for i in tqdm(range(len(q_ids)), desc="Processing questions"):
                # for i in tqdm(range(1), desc="Processing questions"):
                history_lines = self.get_history()
                agent_input = "[question_description]: " + str(q_texts[i])
                if history_lines:
                    agent_input = history_lines + "\n" + agent_input

                print(agent_input)
                response = self.agent_executor_1.invoke({
                    "role": self.role,
                    "task": self.task,
                    "background_knowledge": self.background_knowledge,
                    "tools": self.tool_names,
                    "tool_names": self.tool_names,
                    "final_answer_format": self.final_answer_format,
                    # "input": "[question_description]: " + str(q_texts[i]),
                    "input": agent_input,
                    "agent_scratchpad": ""
                })

                output = response.get("output")
                print(f"{q_ids[i]}: {output}")
                self.save_data(str(q_ids[i]), output,
                               self.config.get("QK_SAVE_PATH").format(model_name=self.config.get("MODEL_NAME")))

                if len(self.history) >= self.history_window:
                    self.history.pop(0)
                self.history.append({
                    "q_id": q_ids[i],
                    "input": q_texts[i],
                    "output": output
                })

                time.sleep(5)

        except Exception as e:
            print(f"[Phase1Executor Error]: {repr(e)}")


# ======================================================================
#      Phase 2: Enriching the Attributes of Knowledge Components
# ======================================================================
class DetermineCognitiveLevelRange(BaseTool):
    name: str = "DetermineCognitiveLevelRange"
    description: str = (
        "Use this tool to determine the continuous range of cognitive levels associated with a given knowledge component. "
        "List all cognitive levels that are relevant for the specified knowledge component, ensuring the selection forms a continuous range. "
        "Provide the 'knowledge_component' parameter, which represents the exact name of the knowledge component (e.g., 'array')."
    )
    llm: Any = Field(default=None)

    def _run(self, knowledge_component: str):
        with open(config.get("KC_RANGE_PROMPT"), "r", encoding="utf-8") as file:
            kc_range_prompt = file.read()

        prompt = kc_range_prompt + "\n" + knowledge_component
        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        return response


class DescribeCognitiveLevelUsage(BaseTool):
    name: str = "DescribeCognitiveLevelUsage"
    description: str = (
        "Use this tool to generate the common uses for each cognitive level associated with a given knowledge component. "
        "Provide the 'cognitive_level_range' parameter, which is a string in the format 'knowledge_component: {cognitive_level_1, cognitive_level_2, ...}', "
        "such as 'array: {Remembering, Understanding, Applying}'. "
        "The tool should output the typical usage of each cognitive level within the context of the specified knowledge component."
    )
    llm: Any = Field(default=None)

    def _run(self, cognitive_level_range: str):
        with open(config.get("KC_USE_PROMPT"), "r", encoding="utf-8") as file:
            kc_use_prompt = file.read()

        prompt = kc_use_prompt + "\n" + cognitive_level_range
        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        return response


class Phase2Executor:
    def __init__(self, llm, role, task, background_knowledge, tools, final_answer_format, config, history_window=5):
        self.llm = llm
        self.role = role
        self.task = task
        self.background_knowledge = background_knowledge
        self.tools = tools
        self.tool_names = ", ".join([tool.name for tool in tools])
        self.final_answer_format = final_answer_format
        self.config = config

        with open(self.config.get("KNOWLEDGE_BASE_TEMPLATE"), "r", encoding="utf-8") as file:
            knowledge_base_template = file.read()

        self.prompt = PromptTemplate(
            input_variables=["role", "task", "background_knowledge", "tools", "tool_names", "final_answer_format",
                             "input", "agent_scratchpad"],
            template=knowledge_base_template,
            validate_template=False
        )

        self.agent_2 = create_react_agent(llm=self.llm, tools=self.tools, prompt=self.prompt)
        self.agent_executor_2 = AgentExecutor(agent=self.agent_2, tools=self.tools, verbose=True,
                                              handle_parsing_errors=True)

        self.history = []
        self.history_window = history_window

    # Return formatted string of previous m rounds (Q&A) for current prompt.
    def get_history(self):
        if not self.history:
            return ""

        history_lines = []
        for idx, rec in enumerate(self.history):
            history_lines.append(
                f"[History round {idx + 1}]\nKnowledge Component (ID: {rec['k_id']}): {rec['input']}\nOutput: {rec['output']}\n"
            )
        return "\n".join(history_lines)

    # Get each knowledge component information.
    def load_data(self, k_file_path):
        data = pd.read_csv(k_file_path)
        k_ids = data["knowledge_component_id"].tolist()
        k_names = data["knowledge_component_name"].tolist()

        return k_ids, k_names

    # Save result ((llm agents_{model_name}) knowledge_cognitive_use.csv).
    def save_data(self, k_id, k_name, response, save_file_path):
        df = pd.DataFrame([(k_id, k_name, response)],
                          columns=["knowledge_component_id", "knowledge_component_name", "cognitive_level_use"])
        header = not os.path.exists(save_file_path)
        df.to_csv(save_file_path, mode="a", header=header, index=False)

    def run(self):
        try:
            k_ids, k_names = self.load_data(self.config.get("KNOWLEDGE_TAGS_FILE_PATH"))
            for i in tqdm(range(len(k_ids)), desc="Processing questions"):
                # for i in tqdm(range(1), desc="Processing questions"):
                history_lines = self.get_history()
                agent_input = "[knowledge_component]: " + str(k_names[i])
                if history_lines:
                    agent_input = history_lines + "\n" + agent_input

                print(agent_input)

                response = self.agent_executor_2.invoke({
                    "role": self.role,
                    "task": self.task,
                    "background_knowledge": self.background_knowledge,
                    "tools": self.tool_names,
                    "tool_names": self.tool_names,
                    "final_answer_format": self.final_answer_format,
                    "input": agent_input,
                    "agent_scratchpad": ""
                })

                output = response.get("output")
                print(f"{k_ids[i]}: {output}")
                self.save_data(str(k_ids[i]), str(k_names[i]), output,
                               self.config.get("KC_USE_SAVE_PATH").format(model_name=self.config.get("MODEL_NAME")))

                if len(self.history) >= self.history_window:
                    self.history.pop(0)
                self.history.append({
                    "k_id": k_ids[i],
                    "input": k_names[i],
                    "output": output
                })

                time.sleep(5)

        except Exception as e:
            print(f"[Phase3Executor Error]: {repr(e)}")


# ======================================================================
#   Phase 3: Determining the Cognitive Level of Knowledge Components
# ======================================================================
class DetermineQuestionKCCognitiveLevel(BaseTool):
    name: str = "DetermineQuestionKCCognitiveLevel"
    description: str = (
        "Use this tool to identify which cognitive level is involved for each knowledge component in a given question. "
        "Provide the 'total_question_information' parameter, which must be a JSON object with the following format: "
        '''
        {
          "question": "The target question text.",
          "knowledge_components": [
            {
              "name": "knowledge component 1",
              "weight": 0.45,
              "cognitive_levels": ["Remembering", "Understanding", "Applying"],
              "common_uses": {
                "Remembering": "Typical usage at remembering level.",
                "Understanding": "Typical usage at understanding level.",
                "Applying": "Typical usage at applying level."
              }
            },
            {
              "name": "knowledge component 2",
              "weight": 0.55,
              "cognitive_levels": ["Remembering", "Understanding"],
              "common_uses": {
                "Remembering": "Typical usage at remembering level.",
                "Understanding": "Typical usage at understanding level."
              }
            }
          ]
        }
        '''
        "The tool should output a JSON object mapping each knowledge component name to the selected cognitive level for this question."
    )
    llm: Any = Field(default=None)

    def _run(self, total_question_information: str):
        with open(config.get("QKC_PROMPT"), "r", encoding="utf-8") as file:
            qkc_prompt = file.read()

        prompt = qkc_prompt + "\n" + total_question_information
        messages = [HumanMessage(content=prompt)]
        response = self.llm.invoke(messages)

        return response


class Phase3Executor:
    def __init__(self, llm, role, task, background_knowledge, tools, final_answer_format, config, history_window=5):
        self.llm = llm
        self.role = role
        self.task = task
        self.background_knowledge = background_knowledge
        self.tools = tools
        self.tool_names = ", ".join([tool.name for tool in tools])
        self.final_answer_format = final_answer_format
        self.config = config

        with open(self.config.get("KNOWLEDGE_BASE_TEMPLATE"), "r", encoding="utf-8") as file:
            knowledge_base_template = file.read()

        self.prompt = PromptTemplate(
            input_variables=["role", "task", "background_knowledge", "tools", "tool_names", "final_answer_format",
                             "input", "agent_scratchpad"],
            template=knowledge_base_template,
            validate_template=False
        )

        self.agent_3 = create_react_agent(llm=self.llm, tools=self.tools, prompt=self.prompt)
        self.agent_executor_3 = AgentExecutor(agent=self.agent_3, tools=self.tools, verbose=True,
                                              handle_parsing_errors=True)

        self.history = []
        self.history_window = history_window

    # Return formatted string of previous m rounds (Q&A) for current prompt.
    def get_history(self):
        if not self.history:
            return ""

        history_lines = []
        for idx, rec in enumerate(self.history):
            history_lines.append(
                f"[History round {idx + 1}]\nQuestion (ID: {rec['q_id']}): {rec['input']}\nOutput: {rec['output']}\n"
            )
        return "\n".join(history_lines)

    # Get each question and its total informations.
    def load_data(self, q_file_path, qk_file_path, kc_use_file_path):
        df_q = pd.read_csv(q_file_path)
        df_qk = pd.read_csv(qk_file_path)
        df_kc_use = pd.read_csv(kc_use_file_path)

        q_ids = df_q["question_id"].tolist()
        q_texts = df_q["description"].tolist()
        qid_text_dict = dict(zip(q_ids, q_texts))

        # Input -- 35: {"array": "0.5", "binary search": "0.4", "hello": "0.l"}
        #   → knowledge_name_set: {"array", "binary search", "hello"}
        #   → knowledge_name_set: {"array", "binary search"}
        #   → knowledge_label： {"array": "0.5", "binary search": "0.4"}
        # Output -- qk_dict: {35: {"array": "0.5", "binary search": "0.4"}, ...}
        qk_dict = {}
        kc_name_set = set(df_kc_use["knowledge_component_name"].tolist())
        for index, row in df_qk.iterrows():
            knowledge_label = json.loads(row["knowledge_component"])

            # Delete knowledge components that do not exist in kc_use_dict
            filtered = {k: v for k, v in knowledge_label.items() if k in kc_name_set}
            qk_dict[row["question_id"]] = filtered

        # kc_range_dict: {"knowledge_component_name": "cognitive_level_range"}
        kc_range_dict = {}
        for _, row in df_kc_use.iterrows():
            kc_name = row["knowledge_component_name"]
            level_use = ast.literal_eval(row["cognitive_level_use"])
            kc_range_dict[kc_name] = list(level_use.keys())

        # kc_use_dict: {"knowledge_component_name": "cognitive_level_use."}
        kc_use_dict = {}
        for index, row in df_kc_use.iterrows():
            kc_use_dict[row["knowledge_component_name"]] = row["cognitive_level_use"]

        # See the format in qkc_prompt.txt for details.
        total_q_information = []
        for q_id, knowledge_label in qk_dict.items():
            q_information = {"question": qid_text_dict.get(q_id), "knowledge_components": []}

            for k_name in set(knowledge_label.keys()):
                q_information["knowledge_components"].append({
                    "name": k_name,
                    "weight": knowledge_label.get(k_name),
                    "cognitive_levels": kc_range_dict.get(k_name),
                    "common_uses": kc_use_dict.get(k_name)
                })

            total_q_information.append(q_information)

        return q_ids, total_q_information

    # Save result ((llm agents_{model_name}) question_knowledge_cognitive.csv).
    def save_data(self, q_id, response, save_file_path):
        df = pd.DataFrame([(q_id, response)], columns=["question_id", "knowledge_component_cognitive"])
        header = not os.path.exists(save_file_path)
        df.to_csv(save_file_path, mode="a", header=header, index=False)

    def run(self):
        try:
            q_ids, total_q_information = self.load_data(
                q_file_path=self.config.get("QUESTION_FILE_PATH"),
                qk_file_path=self.config.get("QK_SAVE_PATH").format(model_name=self.config.get("MODEL_NAME")),
                kc_use_file_path=self.config.get("KC_USE_SAVE_PATH").format(model_name=self.config.get("MODEL_NAME"))
            )

            for i in tqdm(range(len(total_q_information)), desc="Processing questions"):
                # for i in tqdm(range(1), desc="Processing questions"):
                history_lines = self.get_history()
                agent_input = "[total_question_information]: " + str(total_q_information[i])
                if history_lines:
                    agent_input = history_lines + "\n" + agent_input

                print(agent_input)

                response = self.agent_executor_3.invoke({
                    "role": self.role,
                    "task": self.task,
                    "background_knowledge": self.background_knowledge,
                    "tools": self.tool_names,
                    "tool_names": self.tool_names,
                    "final_answer_format": self.final_answer_format,
                    "input": agent_input,
                    "agent_scratchpad": ""
                })

                output = response.get("output")
                print(f"{q_ids[i]}: {output}")
                self.save_data(str(q_ids[i]), output,
                               self.config.get("QKC_SAVE_PATH").format(model_name=self.config.get("MODEL_NAME")))

                if len(self.history) >= self.history_window:
                    self.history.pop(0)
                self.history.append({
                    "q_id": q_ids[i],
                    "input": total_q_information[i],
                    "output": output
                })

                time.sleep(5)

        except Exception as e:
            print(f"[Phase3Executor Error]: {repr(e)}")


# ======================================================================================
#                                     Main
# ======================================================================================
def main(config_file_path, dataset_name):
    set_config(config_file_path, dataset_name)
    llm = set_llm(config.get("MODEL_NAME"))

    phase_1_executor = Phase1Executor(
        llm=llm,
        role="Educational knowledge analysis expert.",
        task=(
            "You are given a [question_description]. "
            "Your task is to identify the most relevant knowledge components (1 to 5), and assign each a numeric weight reflecting its relative importance. "
            "When calling the tool IdentifyQuestionKnowledgeComponents, always use the original question text as the 'question_description' parameter, without any summarization or modification."
        ),
        background_knowledge=(
            "**Knowledge Component for Computer Science**:\n"
            "[string, two pointers, bit manipulation, prefix sum, hash function, rolling hash, database, array, tree array, linked list, doubly-linked list, hash table, tree, binary tree, trie, segment tree, binary search tree, heap, stack, monotonic stack, queue, monotonic queue, graph, union find, ordered set, recursion, enumeration, divide and conquer, binary search, topological sort, merge sort, counting sort, bucket sort, dynamic programming, memorization search, greedy, backtracking, depth-first search, breadth-first search, sliding window, state compression, string matching, shortest path, minimum spanning tree, design, simulation, search, sort, union set, mathematics, loop statement, number theory, pascal's triangle, matrix, combinatorics, branch, game, 01 backpack, binary exponentiation, iteration, big integer, prime number, fibonacci sequence, pointer, macro, struct, if statement, brute force, dynamic allocation, array of structures, radix sort, quick sort, hash map, heap sort, top-k, kruskal, circular queue, circular linked list, binary graph, operator, library function]\n"
            "**Knowledge Component for Mathematics**:\n"
            "[Decimals, Factors, Multiples and Primes, Fractions, Percentages, Proportion, Ratio, Expanding Brackets, Factorising, Straight Line Graphs, Quadratic Graphs, Graphs of Trigonometric Functions, Inequalities, Sequences, Simultaneous Equations, Linear Equations, Quadratic Equations, Substitution into Formula, Transformation of Functions, Angles, Area of Simple Shapes, Bearings, Circle Theorems, Co-ordinates, Properties of Polygons, Perimeter, Properties of Quadrilaterals, Symmetry, Reflection, Rotation, Enlargement, Translation and Vectors, Basic Trigonometry, Units of Measurement, Volume and Surface Area, Box Plots, Histogram, Pie Chart, Probability of Single Events, Data Collection, Sampling and Bias, Scatter Diagram, Time Series and Line Graphs, Matrices, Solving Equations, Formula, Equation of a Circle, Linear Sequences (nth term), Quadratic Sequences, Measuring Angles, Basic Angle Facts (straight line, opposite, around a point, etc), Angle Facts with Parallel Lines, Angles in Polygons, Parts of a Circle, Circumference, Area of a Circle, Sectors of a Circle, Volume of Prisms, Construct Triangle, Length Units, Area Units, Volume and Capacity Units, Weight Units, Rounding to Decimal Places, Rounding to Significant Figures, Estimation, Prime Numbers and Prime Factors, Equivalent Fractions, Simplifying Fractions, Converting Mixed Number and Improper Fractions, Adding and Subtracting Fractions, Multiplying Fractions, Dividing Fractions, Percentage Increase and Decrease, Squares, Cubes, etc, Square Roots, Cube Roots, etc, Laws of Indices, Simplifying Surds, Rationalising the Denominator, Simplifying Expressions by Collecting Like Terms, Multiplying Terms, Dividing Terms, Difference of Two Squares, Algebraic Fractions, Completing the Square, Plotting Lines from Tables of Values, Finding the Equation of a Line, Parallel Lines, Perpendicular Lines, Sketching from Factorised Form, Graphical Solution of Quadratic Equations, Speed, Distance, Time, Density, Pythagoras, Similarity and Congruency, Line Symmetry, Right-angled Triangles (SOHCAHTOA), Non Right-angled Triangles (Sine and Cosine Rules), Area of Non Right-angled Triangles, Permutations and Combinations, Tally Charts, Frequency Diagram and Frequency Polygon, Venn Diagrams, Probability, Tree Diagrams with Independent Events, Arithmetic Sequences, Sigma Notation, Summing Series, Basic dy/dx, Differentiation from First Principals, Finding Maximums and Minimums, Integration, Radians, Solving Basic Trigonometric Equations, Basic Trigonometric Identities, Laws of Logarithms, Solving Equations with Exps and Logs, Domain and Range, Inverse Functions, Composite Functions, Standard Deviation and Variance from Discrete Data, Factorial Notation]\n"
        ),
        tools=[IdentifyQuestionKnowledgeComponents(llm=llm)],
        final_answer_format="""{"knowledge component 1": weight 1, "knowledge component 2": weight 2}""",
        config=config,
        history_window=5
    )

    phase_2_executor = Phase2Executor(
        llm=llm,
        role="Educational cognitive analysis expert.",
        task=(
            "You are given a [knowledge component]. "
            "Your task is to: "
            "1) Determine the continuous range of cognitive levels (abilities) involved in this knowledge component. "
            "2) For each identified cognitive level, describe its common uses in the context of the given knowledge component. "
            "When calling the tool DetermineCognitiveLevelRange, always use the original knowledge component name as the 'knowledge_component' parameter, without any summarization or modification. "
            "When calling the tool DescribeCognitiveLevelUsage, always provide the knowledge component and its cognitive levels in the format 'knowledge_component: {cognitive_level_1, cognitive_level_2, ...}' as the 'cognitive_level_range' parameter, exactly as obtained from the previous step, without any alteration."
        ),
        background_knowledge=(
            "**Cognitive Level Definitions**:\n"
            "**Remembering**: The first level of cognitive levels, focusing on recalling basic definitions and attributes of the knowledge component. It often involves direct recollection or simple enumeration in applications.\n"
            "**Understanding**: The second level of cognitive levels, involving common operations and basic applications, such as demonstrating basic operations and explaining how they work.\n"
            "**Applying**: The third level of cognitive levels, involving the use of knowledge to solve practical problems or construct solutions in new scenarios. This level emphasizes the application of knowledge to specific real-world situations.\n"
            "**Analyzing**: The fourth level of cognitive levels, with higher requirements for the use of the knowledge component, focusing on deep analysis to optimize solutions or improve methods.\n"
            "**Evaluating**: The fifth level of cognitive levels requires using the knowledge component to be assessed based on specific criteria (effectiveness, reliability, efficiency, etc.). This level emphasizes assessing the performance of the knowledge component in theoretical and practical applications, and making decisions on whether to continue using, improve, or replace it.\n"
            "**Creating**: The highest level of cognitive levels. It requires integrating the knowledge component with knowledge from other fields to propose creative solutions for meeting needs or solving complex practical problems. This level focuses on creativity and innovation.\n"
        ),
        tools=[DetermineCognitiveLevelRange(llm=llm), DescribeCognitiveLevelUsage(llm=llm)],
        final_answer_format="""{"cognitive level 1": "usage 1",  "cognitive level 2": "usage 2"}""",
        config=config,
        history_window=5
    )

    phase_3_executor = Phase3Executor(
        llm=llm,
        role="Educational cognitive analysis expert.",
        task=(
            "You are given a question and its detailed information. "
            "Your task is to identify which cognitive level is involved for each knowledge component in the given question. "
            "When calling the tool DetermineQuestionKCCognitiveLevel, always provide the 'total_question_information' parameter, without any summarization or modification."
        ),
        background_knowledge=(
            "**Cognitive Level Definitions**:\n"
            "**Remembering**: The first level of cognitive levels, focusing on recalling basic definitions and attributes of the knowledge component. It often involves direct recollection or simple enumeration in applications.\n"
            "**Understanding**: The second level of cognitive levels, involving common operations and basic applications, such as demonstrating basic operations and explaining how they work.\n"
            "**Applying**: The third level of cognitive levels, involving the use of knowledge to solve practical problems or construct solutions in new scenarios. This level emphasizes the application of knowledge to specific real-world situations.\n"
            "**Analyzing**: The fourth level of cognitive levels, with higher requirements for the use of the knowledge component, focusing on deep analysis to optimize solutions or improve methods.\n"
            "**Evaluating**: The fifth level of cognitive levels requires using the knowledge component to be assessed based on specific criteria (effectiveness, reliability, efficiency, etc.). This level emphasizes assessing the performance of the knowledge component in theoretical and practical applications, and making decisions on whether to continue using, improve, or replace it.\n"
            "**Creating**: The highest level of cognitive levels. It requires integrating the knowledge component with knowledge from other fields to propose creative solutions for meeting needs or solving complex practical problems. This level focuses on creativity and innovation.\n"
        ),
        tools=[DetermineQuestionKCCognitiveLevel(llm=llm)],
        final_answer_format="""{"knowledge component 1": cognitive level 1, "knowledge component 2": cognitive level 2}""",
        config=config,
        history_window=5
    )

    phase_1_executor.run()
    phase_2_executor.run()
    phase_3_executor.run()


if __name__ == "__main__":
    main(config_file_path="./drive/MyDrive/llm_agents/data/data_config.json", dataset_name="bepkt")
